# Analyze PCAPs

## Distance between Frames

In [ ]:
from dpkt.pcap import Reader
from scipy import stats
from numpy import median
import seaborn as sns
sns.set_theme()

In [ ]:
input_filename = "/home/arch/testbed_src/src/pcap_writer/videoBBB.pcap"
reader = Reader(open(input_filename, "rb"))

timestamps = []
for ts, buf in reader:
    timestamps.append(ts)
    
ifg = []

last = 0
for ts in timestamps:
    ifg.append(float(ts - last))
    last = ts

In [ ]:
print(stats.describe(ifg))
print(median(ifg))

In [ ]:
ax = sns.ecdfplot(ifg, complementary=False, stat="proportion", log_scale=True)
ax.set(xlabel='Distance between Frames [µs]', ylabel='ECDF', title="BBB Video [h264,1080p,60fps]")
plt.savefig("videoBBB-ecdf.png", dpi=300)
plt.show()

In [ ]:
ax = sns.histplot(ifg, stat="percent", log_scale=True)
ax.set(xlabel='Distance between Frames [µs]', ylabel='Percent', title="BBB Video [h264,1080p,60fps]")
plt.savefig("videoBBB-hist.png", dpi=300)
plt.show()

## Frame Size

In [ ]:
from dpkt.pcap import Reader
from dpkt.ethernet import Ethernet
from pandas import DataFrame, Series

In [ ]:
files = ['podcast',
         'robotic',
         'audio',
         'spotJPEGhost',
         'spotJPEGspot',
         'spotRAWhost',
         'spotRAWspot',
         'spotWASDhost',
         'spotWASDspot',
         'tactileGLOVE',
         'videoBBB']

path = "/home/arch/testbed_src/src/pcap_writer/"

file_path = ["{}{}.pcap".format(path, f) for f in files]

In [ ]:
frame_sizes_all = {}
for file, path in zip(files, file_path):
    reader = Reader(open(path, "rb"))
    frame_sizes = []
    for ts, buf in reader:
        eth = Ethernet(buf)
        frame_sizes.append(len(eth))
    frame_sizes_all[file] = frame_sizes
df = DataFrame(dict([(k, Series(v)) for k, v in frame_sizes_all.items()]))

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html
> Descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.

In [ ]:
stats = df.describe()

In [ ]:
stats.to_csv('input_pcaps_frame_sizes.csv')

In [ ]:
stats